In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import numpy as np
import os

from Software.GenuVP import runGNVP as gnvp

from PlaneDefinition.plane import Airplane as Plane
from PlaneDefinition.wing import Wing as wg
import PlaneDefinition.wing as wing

from Flight_Dynamics.disturbances import disturbance as disturb

from Database.getresults import Database_2D
from Database import DB3D , BASEGNVP

In [ ]:
HOMEDIR = os.getcwd()

# Airfoil Data

In [ ]:
db = Database_2D(HOMEDIR)
airfoils = db.getAirfoils()
polars2D = db.Data

# Get Plane

In [ ]:
Origin = np.array([0., 0., 0.])

In [ ]:
wingPos = np.array([0.0,0.0,0.0])
wingOrientation = np.array([0.0,0.0,0.0])

mainWing = wg(name="wing",
              airfoil=airfoils['NACA4415'],
              Origin=Origin + wingPos,
              Orientation=wingOrientation,           
              isSymmetric=True,
              span=2 * 1.130,
              sweepOffset=0,
              dihAngle=0,
              chordFun=wing.linearChord,
              chord=[0.159, 0.072],
              spanFun=wing.linSpan,
              N = 20,
              M = 13,
              mass = 0.670)
# mainWing.plotWing()

In [ ]:
elevatorPos = np.array([0.54,0.,0.])
elevatorOrientantion = np.array([0.,0.,0.])

elevator = wg(name = "tail",
              airfoil=airfoils['NACA0008'],
              Origin=Origin+ elevatorPos,
              Orientation=elevatorOrientantion,
              isSymmetric=True,
              span= 2 * 0.169,
              sweepOffset=0,
              dihAngle=0,
              chordFun=wing.linearChord,
              chord=[0.130, 0.03],
              spanFun=wing.linSpan,
              N = 15,
              M = 8,
              mass = 0.06)
# elevator.plotWing()

In [ ]:
rudderPos = np.array([0.54, 0., 0.01])
rudderOrientantion = np.array([0.0,0.0,90.0])

rudder = wg(name= "rudder",
            airfoil=airfoils['NACA0008'],
            Origin=Origin + rudderPos,
            Orientation= rudderOrientantion,
            isSymmetric=False,
            span= 0.165,
            sweepOffset=0.1,
            dihAngle=0,
            chordFun=wing.linearChord,
            chord=[0.2, 0.1],
            spanFun=wing.linSpan,
            N = 15,
            M = 8,
            mass = 0.04)
# rudder.plotWing()

In [ ]:
liftingSurfaces = [mainWing, elevator, rudder]
ap = Plane("Plane", liftingSurfaces)
ap.accessDB(HOMEDIR, DB3D)
# ap.visAirplane()


In [ ]:
cleaning = False
calcGenu = True
calcBatchGenu = True
petrubationAnalysis = True

## AoA Run

In [ ]:
AoAmax = 10
AoAmin = -6
NoAoA = (AoAmax - AoAmin) + 1
angles = np.linspace(AoAmin, AoAmax, NoAoA)

Uinf = 20

In [ ]:
if calcBatchGenu == True:
    genuBatchArgs = [ap, BASEGNVP, polars2D,"Xfoil",Uinf, angles]
    # ap.runSolver(gnvp.runGNVPangles,genuBatchArgs)
genuPolarArgs = [ap.CASEDIR, HOMEDIR]
ap.makePolars(gnvp.makePolar,genuPolarArgs)
ap.defineSim(Uinf, 1.225)
ap.save()


# Dynamics

In [ ]:
from Flight_Dynamics.dyn_plane import dyn_plane as dp

### Define and Trim Plane

In [ ]:
ap.M = 3 ## HAS TO BE DEFINED SINCE I HAVE NOT ADDED MASSED
dyn = dp(ap,polars2D)

### Pertrubations

In [ ]:
dyn.allPerturb(5e-2, "Central")
# dyn.get_pertrub()

In [ ]:
if petrubationAnalysis == True:
    genuBatchArgs = [dyn, BASEGNVP, polars2D,"Xfoil",dyn.trim['U'], dyn.trim['AoA']]
    dyn.accessDB(HOMEDIR)
    # dyn.runAnalysis(gnvp.runGNVPpertr,genuBatchArgs)
genuLogArgs = [dyn.DynDir, HOMEDIR]
dyn.logResults(gnvp.logResults,genuLogArgs)
dyn.save()

In [ ]:
dyn.scheme = "Central"
dyn.longitudalStability()
dyn.lateralStability()


In [ ]:
dyn.Along
# [xu, xw, xq, xth]
# [zu, zw, zq, zth]
# [mu, mw, mq, mth]
# [0,  1 , 0 ,  0 ]

# dyn.AstarLong

In [ ]:
dyn.Alat
# [yv, yp , yr, yphi]
# [lv, lp , lr, lphi]
# [nv, n_p, nr, nph ]
# [0 , 1  , 0 , 0   ]

# dyn.AstarLat